In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
import ast

In [ ]:
im_src_dir = '../raw_image/all/'    # The folder where the original image is located
translate_file_path = 'matrix.xlsx' # Table for storing transformation matrices
dataset_path = '../dataset/'        # The save path for the generated dataset

In [ ]:
# Perform 72 transformations on an original image and save it to the corresponding folder
def transform(_img_path,_output_dir,translate_file_path):
    # Read transformation matrix table
    data = pd.read_excel(translate_file_path)
    data4trans = data[['trans_type','direction','degree','trans_matrix','roi']]

    # Reading image
    im_src = cv2.imread(_img_path)
    # Save a separate copy of the original image and its cropped image
    if not os.path.exists(_output_dir+'src/'):
        os.mkdir(_output_dir+'src/')
    cv2.imwrite(_output_dir+'src/im_src.png',im_src)
    roi = np.int32(str2mtx(data4trans['roi'][0]))
    im_src_clip = im_src[roi[0]:roi[1],roi[2]:roi[3]]
    cv2.imwrite(_output_dir+'src/im_src_clip.png',im_src_clip)

    # Generate transformation graphs for each transformation matrix
    rows = data4trans.shape[0]
    for i in range(0,rows):
        # Read transformation matrix
        h = np.float64(str2mtx(data4trans['trans_matrix'][i]))
        # Perform transformation
        im_create = cv2.warpPerspective(im_src, h, (im_src.shape[1],im_src.shape[0]))
        # Create corresponding folder
        output_dir = _output_dir+str(data4trans['trans_type'][i])+'_'+str(data4trans['direction'][i])+'_'+str(data4trans['degree'][i]).rjust(3,'0')+'/'
        print(output_dir)
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)


        # Save Transformation Matrix
        np.savetxt(output_dir+'h.txt',h)
        # Save the transformed image
        cv2.imwrite(output_dir+'im_dst.png',im_create)
        # Read the coordinates of the captured area
        roi = np.int32(str2mtx(data4trans['roi'][i]))
        # Cropping
        im_create_clip = im_create[roi[0]:roi[1],roi[2]:roi[3]]
        # Save the coordinates of the captured area
        np.savetxt(output_dir+'roi.txt',roi)
        # Save the screenshot of the transformed image
        cv2.imwrite(output_dir+'im_dst_clip.png',im_create_clip)

# Convert string to 2D array
def str2mtx(input_str):
    # Remove commas and split into one-dimensional arrays
    input_str = input_str.replace(' ','')
    list_str = input_str.replace(',', ' ').split()
    # Splicing a one-dimensional array into a two-dimensional array
    two_dim_list = ast.literal_eval('[' + ','.join(list_str) + ']')
    res = two_dim_list[0]
    return res

In [ ]:
# Create a folder corresponding to each image
files = os.listdir(im_src_dir)

for file in files:
    print(file)
    if not os.path.exists(dataset_path+file.split('.')[0]+'/'):
        os.mkdir(dataset_path+file.split('.')[0]+'/')

In [ ]:
# Perform transformations on each image and save them
for file in files:
    output_dir = dataset_path+file.split('.')[0]+'/'
    print(output_dir)
    transform(im_src_dir+file,output_dir,translate_file_path)